In [1]:
import tensorflow as tf
from tensorflow.contrib import predictor

export_dir = "model_sr50"
predict_fn = predictor.from_saved_model(export_dir)

INFO:tensorflow:Restoring parameters from model_sr50/variables/variables


In [2]:
from matplotlib import pyplot as plt
import numpy as np
import random

In [3]:
import sys 
sys.path.insert(0,'../..')

In [4]:
from cnf_dataset import clauses_to_matrix
from dpll import DPLL, RandomClauseDPLL, MostCommonVarDPLL, RandomVarDPLL
from cnf import get_random_kcnf, CNF, get_sats_SR, get_pos_SR
from tqdm import tqdm
from collections import Counter

In [5]:
import math
from collections import defaultdict

def jw(clauses):
    score = defaultdict(int)

    for clause in clauses:
        for l in clause:
            score[l] += 2. ** (-len(clause))

    return max(score, key=score.get)


In [6]:
LIMIT_RUNS = 1000000

In [7]:
def shorten_cnf(cnf: CNF):
    for c in cnf.clauses:
        if len(c) == 1:
            return shorten_cnf(cnf.set_var(c[0]))
    all_literals = set(x
                       for clause in cnf.clauses
                       for x in clause)
    for v in cnf.vars:
        if v in all_literals and (-v) not in all_literals:
            return shorten_cnf(cnf.set_var(v))
        if (-v) in all_literals and v not in all_literals:
            return shorten_cnf(cnf.set_var(-v))
    return cnf

def make_normalized(cls):
    class NormalizedDPLL(cls):
        def run(self, cnf: CNF):
            assert isinstance(cnf, CNF)
            self.number_of_runs += 1
            if self.number_of_runs > LIMIT_RUNS:
                return None
            
            cnf = shorten_cnf(cnf)
            if cnf.is_true():
                return []
            elif cnf.is_false():
                return None

            sug_var = self.suggest(cnf)
            sug_cnf = cnf.set_var(sug_var)

            sug_res = self.run(sug_cnf)
            if sug_res is not None:
                return [sug_var] + sug_res

            not_sug_cnf = cnf.set_var(-sug_var)
            not_sug_res = self.run(not_sug_cnf)
            if not_sug_res is not None:
                self.number_of_errors += 1
                return [-sug_var] + not_sug_res
            return None
    NormalizedDPLL.__name__ = "Normalized{}".format(cls.__name__)
    return NormalizedDPLL

In [8]:
np.set_printoptions(precision=3, suppress=True)

In [9]:
import tensorflow as tf
import os

BATCH_SIZE = 1

In [10]:

class GraphBasedDPLL(DPLL):
    def suggest(self, input_cnf: CNF):
        clause_num = len(input_cnf.clauses)
        var_num = max(input_cnf.vars)
        inputs = np.asarray([clauses_to_matrix(input_cnf.clauses, clause_num, var_num)] * BATCH_SIZE)
        
        policy_probs = predict_fn({"input": inputs})['policy_probabilities']
        
        best_prob = 0.0
        best_svar = None
        for var in input_cnf.vars:
            for svar in [var, -var]:
                svar_prob = policy_probs[0][var-1][0 if svar > 0 else 1]
                if svar_prob > best_prob:
                    best_prob = svar_prob
                    best_svar = svar
        return best_svar

class MostCommonDPLL(DPLL):
    def suggest(self, cnf: CNF):
        counter = Counter()
        for clause in cnf.clauses:
            for svar in clause:
                counter[svar] += 1
        return counter.most_common(1)[0][0]
    
class JeroslawDPLL(DPLL):
    def suggest(self, cnf: CNF):
        return jw(cnf.clauses)
    

class HumbleDPLL(DPLL):
    def suggest(self, input_cnf: CNF):
        clause_num = len(input_cnf.clauses)
        var_num = max(input_cnf.vars)
        inputs = np.asarray([clauses_to_matrix(input_cnf.clauses, clause_num, var_num)] * BATCH_SIZE)
        
        output = predict_fn({"input": inputs})
        policy_probs = output['policy_probabilities']
        sat_prob = output['sat_probabilities'][0]
        
        best_prob = 0.0
        best_svar = None
        for var in input_cnf.vars:
            for svar in [var, -var]:
                svar_prob = policy_probs[0][var-1][0 if svar > 0 else 1]
                if svar_prob > best_prob:
                    best_prob = svar_prob
                    best_svar = svar
        if sat_prob < 0.3:
            # Overwriting with JW
            best_svar = jw(input_cnf.clauses)
        return best_svar    

In [11]:
NormalizedGraphBasedDPLL = make_normalized(GraphBasedDPLL)
NormalizedMostCommonDPLL = make_normalized(MostCommonDPLL)
NormalizedJeroslawDPLL = make_normalized(JeroslawDPLL)
NormalizedHumbleDPLL = make_normalized(HumbleDPLL)

In [12]:
class FastHumbleDPLL(DPLL):
    def run(self, cnf: CNF, fast=False):
        assert isinstance(cnf, CNF)
        self.number_of_runs += 1
        if self.number_of_runs > LIMIT_RUNS:
            return None

        cnf = shorten_cnf(cnf)
        if cnf.is_true():
            return []
        elif cnf.is_false():
            return None

        switch_to_heuristic, sug_var = self.suggest(cnf, fast)
        sug_cnf = cnf.set_var(sug_var)

        sug_res = self.run(sug_cnf, switch_to_heuristic)
        if sug_res is not None:
            return [sug_var] + sug_res

        not_sug_cnf = cnf.set_var(-sug_var)
        not_sug_res = self.run(not_sug_cnf, switch_to_heuristic)
        if not_sug_res is not None:
            self.number_of_errors += 1
            return [-sug_var] + not_sug_res
        return None

    def suggest(self, input_cnf: CNF, fast):
        if not fast:
            clause_num = len(input_cnf.clauses)
            var_num = max(input_cnf.vars)
            inputs = np.asarray([clauses_to_matrix(input_cnf.clauses, clause_num, var_num)] * BATCH_SIZE)

            output = predict_fn({"input": inputs})
            policy_probs = output['policy_probabilities']
            sat_prob = output['sat_probabilities'][0]

            best_prob = 0.0
            best_svar = None
            for var in input_cnf.vars:
                for svar in [var, -var]:
                    svar_prob = policy_probs[0][var-1][0 if svar > 0 else 1]
                    #print(svar, svar_prob, best_prob, file=logfile)
                    if svar_prob > best_prob:
                        best_prob = svar_prob
                        best_svar = svar
        if fast or sat_prob < 0.3:
            # Overwriting with JW
            best_svar = jw(input_cnf.clauses)
        return (fast or sat_prob < 0.3), best_svar    

In [13]:
def compute_steps(sats, dpll_cls):
    steps = []
    errors = []
    solved = 0
    for sat in tqdm(sats):
        dpll = dpll_cls()
        res = dpll.run(sat)
        if res is not None:
            steps.append(dpll.number_of_runs)
            errors.append(dpll.number_of_errors)
            solved += 1
    print("Within {} steps solved {} problems out of {}".format(LIMIT_RUNS, solved, len(sats)))
    return steps, errors

In [14]:
def compute_and_print_steps(sats, dpll_cls):
    print(dpll_cls.__name__)
    steps, errors = compute_steps(sats, dpll_cls)
    print("#Sats: {}; avg step: {:.2f}; stdev step: {:.2f}; avg error: {:.2f}; stdev error: {:.2f}".format(
        len(steps), np.mean(steps), np.std(steps), np.mean(errors), np.std(errors)))
    print("Table: ", steps)

In [15]:
def print_all(s, n, m, light=False):
    global S, N, M
    S = s
    N = n # number of clauses
    M = m # number of variables
    
    MAX_TRIES = 100000
    sats = []
    
    random.seed(1)
    np.random.seed(1)
    
    for index in range(MAX_TRIES):
        if len(sats) >= S:
            break
        sat = get_pos_SR(M, M, N)
        sats.append(sat)
    assert len(sats) == S
    print("We have generated {} formulas".format(len(sats)))
    compute_and_print_steps(sats, FastHumbleDPLL)
    compute_and_print_steps(sats, NormalizedJeroslawDPLL)

In [16]:
print_all(10, 1000, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

We have generated 10 formulas
FastHumbleDPLL


100%|██████████| 10/10 [00:00<00:00, 823.83it/s]

Within 1000000 steps solved 10 problems out of 10
#Sats: 10; avg step: 5.30; stdev step: 1.10; avg error: 0.00; stdev error: 0.00
Table:  [7, 5, 4, 7, 4, 6, 6, 4, 5, 5]
NormalizedJeroslawDPLL
Within 1000000 steps solved 10 problems out of 10
#Sats: 10; avg step: 6.00; stdev step: 1.48; avg error: 0.30; stdev error: 0.46
Table:  [8, 6, 9, 4, 5, 6, 7, 5, 5, 5]


In [17]:
print_all(100, 200, 10)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


100%|██████████| 100/100 [00:00<00:00, 636.47it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 5.54; stdev step: 1.85; avg error: 0.03; stdev error: 0.17
Table:  [7, 5, 4, 7, 4, 6, 6, 4, 5, 5, 5, 5, 5, 5, 5, 8, 6, 5, 6, 7, 7, 3, 7, 4, 6, 6, 5, 5, 12, 6, 4, 3, 5, 5, 5, 5, 6, 3, 5, 12, 6, 3, 4, 5, 7, 6, 4, 5, 4, 4, 4, 4, 7, 4, 4, 7, 6, 5, 6, 6, 6, 5, 7, 5, 6, 5, 5, 6, 4, 6, 7, 6, 7, 6, 6, 4, 4, 6, 7, 17, 5, 6, 4, 5, 4, 5, 7, 5, 5, 5, 4, 6, 5, 5, 3, 6, 6, 5, 6, 7]
NormalizedJeroslawDPLL
Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 7.31; stdev step: 3.60; avg error: 0.49; stdev error: 0.56
Table:  [8, 6, 9, 4, 5, 6, 7, 5, 5, 5, 15, 8, 5, 3, 6, 5, 6, 5, 11, 4, 5, 22, 7, 5, 6, 13, 6, 15, 11, 6, 5, 3, 7, 7, 5, 5, 5, 5, 6, 12, 16, 5, 4, 7, 8, 3, 4, 3, 5, 7, 6, 12, 12, 18, 9, 6, 8, 7, 6, 7, 6, 5, 7, 11, 4, 10, 7, 4, 6, 4, 7, 6, 10, 5, 8, 7, 11, 7, 5, 11, 6, 7, 5, 8, 7, 15, 6, 4, 12, 8, 5, 20, 9, 6, 6, 5, 10, 5, 5, 4]


In [18]:
print_all(100, 200, 20)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


 13%|█▎        | 13/100 [00:00<00:00, 118.86it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 14.81; stdev step: 14.54; avg error: 0.19; stdev error: 0.42
Table:  [8, 9, 6, 5, 10, 7, 10, 7, 9, 24, 8, 11, 8, 7, 7, 11, 12, 34, 43, 12, 8, 8, 8, 14, 11, 9, 61, 37, 7, 9, 7, 50, 11, 11, 9, 14, 10, 7, 12, 16, 37, 7, 13, 9, 85, 16, 12, 8, 23, 11, 10, 6, 8, 9, 31, 9, 10, 10, 10, 11, 8, 10, 44, 12, 35, 9, 7, 5, 21, 85, 9, 8, 11, 4, 8, 8, 8, 11, 7, 31, 8, 9, 9, 8, 14, 10, 9, 11, 10, 10, 6, 8, 13, 12, 6, 9, 24, 12, 42, 8]
NormalizedJeroslawDPLL


100%|██████████| 100/100 [00:00<00:00, 123.67it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 18.68; stdev step: 11.47; avg error: 1.14; stdev error: 0.74
Table:  [7, 25, 14, 8, 20, 9, 6, 20, 27, 42, 28, 6, 34, 6, 23, 10, 31, 23, 14, 11, 8, 8, 9, 14, 21, 6, 7, 25, 6, 9, 16, 31, 9, 9, 6, 19, 10, 13, 8, 12, 30, 10, 20, 26, 24, 32, 9, 26, 22, 21, 21, 11, 27, 32, 30, 18, 18, 22, 28, 37, 13, 10, 33, 11, 38, 22, 8, 29, 32, 63, 8, 13, 5, 5, 8, 10, 7, 14, 8, 11, 21, 34, 9, 11, 40, 11, 7, 47, 25, 17, 9, 8, 41, 21, 27, 44, 19, 9, 32, 14]


In [19]:
print_all(100, 500, 30)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


 10%|█         | 10/100 [00:00<00:02, 41.00it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 26.60; stdev step: 39.70; avg error: 0.20; stdev error: 0.42
Table:  [7, 76, 42, 13, 15, 89, 16, 15, 6, 12, 11, 12, 9, 12, 17, 21, 105, 11, 9, 9, 106, 15, 17, 14, 28, 13, 12, 12, 193, 15, 13, 8, 9, 15, 15, 106, 7, 13, 68, 15, 14, 9, 128, 10, 9, 17, 12, 9, 11, 12, 73, 9, 9, 205, 80, 50, 9, 9, 11, 13, 13, 10, 15, 10, 8, 19, 11, 11, 7, 16, 7, 198, 15, 13, 9, 9, 7, 14, 74, 11, 11, 14, 13, 17, 10, 9, 10, 10, 10, 9, 10, 11, 7, 9, 88, 11, 9, 6, 10, 9]
NormalizedJeroslawDPLL


100%|██████████| 100/100 [00:02<00:00, 34.09it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 32.71; stdev step: 25.86; avg error: 1.23; stdev error: 0.80
Table:  [16, 57, 17, 21, 17, 14, 56, 16, 10, 69, 14, 32, 17, 52, 19, 14, 8, 11, 9, 62, 32, 17, 78, 28, 28, 32, 26, 22, 8, 30, 145, 17, 6, 44, 12, 80, 28, 52, 62, 14, 37, 32, 41, 49, 27, 37, 40, 34, 64, 33, 34, 14, 17, 87, 62, 82, 10, 15, 11, 39, 58, 75, 59, 56, 11, 58, 12, 16, 13, 14, 10, 90, 41, 16, 45, 12, 8, 47, 10, 25, 11, 67, 63, 15, 7, 11, 6, 12, 10, 77, 29, 8, 14, 29, 101, 13, 37, 10, 10, 8]


In [20]:
print_all(100, 10000, 40, light=True)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


  1%|          | 1/100 [00:00<00:13,  7.46it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 59.92; stdev step: 105.82; avg error: 0.41; stdev error: 0.68
Table:  [12, 17, 13, 16, 16, 34, 16, 14, 17, 21, 331, 139, 14, 18, 11, 14, 7, 16, 295, 15, 78, 263, 83, 17, 14, 16, 11, 18, 31, 13, 22, 12, 179, 14, 12, 18, 14, 17, 22, 18, 20, 17, 17, 24, 22, 13, 11, 12, 13, 12, 537, 18, 16, 19, 10, 16, 15, 439, 20, 14, 9, 10, 52, 589, 19, 15, 15, 61, 16, 33, 19, 25, 20, 11, 13, 137, 171, 18, 119, 11, 96, 30, 14, 14, 133, 27, 12, 13, 16, 243, 76, 192, 248, 168, 18, 13, 11, 14, 12, 106]
NormalizedJeroslawDPLL


100%|██████████| 100/100 [00:11<00:00,  4.62it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 83.80; stdev step: 73.45; avg error: 1.84; stdev error: 1.12
Table:  [88, 184, 145, 12, 12, 78, 128, 100, 39, 63, 18, 108, 103, 16, 23, 20, 28, 41, 186, 82, 36, 82, 30, 54, 98, 17, 60, 12, 67, 12, 49, 290, 86, 66, 21, 65, 14, 156, 177, 20, 26, 109, 32, 31, 34, 119, 91, 16, 48, 10, 14, 25, 10, 25, 88, 288, 107, 301, 11, 13, 17, 15, 219, 227, 14, 46, 35, 132, 105, 95, 142, 156, 54, 54, 19, 220, 155, 18, 96, 49, 112, 18, 44, 15, 180, 48, 13, 123, 63, 262, 76, 138, 142, 188, 16, 98, 248, 228, 45, 171]


In [21]:
print_all(100, 10000, 50, light=True)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


  3%|▎         | 3/100 [00:00<00:15,  6.30it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 113.30; stdev step: 187.64; avg error: 0.53; stdev error: 0.82
Table:  [15, 18, 23, 28, 19, 17, 24, 20, 22, 13, 19, 21, 812, 122, 19, 18, 280, 23, 401, 348, 35, 25, 15, 21, 35, 23, 41, 22, 23, 19, 16, 18, 15, 14, 27, 670, 26, 106, 15, 61, 23, 55, 18, 54, 20, 17, 18, 70, 361, 17, 16, 32, 19, 21, 11, 617, 29, 695, 20, 19, 19, 83, 12, 16, 16, 13, 548, 13, 430, 21, 384, 20, 20, 16, 21, 17, 249, 626, 133, 13, 524, 493, 7, 24, 207, 26, 17, 8, 18, 15, 25, 18, 538, 25, 13, 293, 328, 120, 20, 320]
NormalizedJeroslawDPLL


100%|██████████| 100/100 [00:34<00:00,  2.57it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 152.69; stdev step: 146.97; avg error: 2.01; stdev error: 1.09
Table:  [34, 18, 210, 83, 121, 322, 143, 14, 249, 17, 24, 127, 487, 242, 603, 214, 131, 499, 20, 174, 75, 42, 36, 13, 35, 56, 159, 77, 499, 67, 117, 36, 12, 77, 27, 621, 258, 68, 86, 19, 237, 177, 171, 154, 35, 424, 130, 317, 671, 204, 16, 40, 11, 114, 54, 92, 108, 50, 18, 16, 225, 33, 58, 72, 164, 12, 56, 18, 222, 46, 20, 59, 19, 153, 245, 105, 189, 296, 396, 14, 344, 269, 44, 230, 284, 55, 145, 258, 331, 18, 121, 141, 319, 327, 73, 151, 332, 26, 92, 156]


In [22]:
print_all(100, 10000, 70, light=True)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


  1%|          | 1/100 [00:04<08:05,  4.90s/it]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 1050.37; stdev step: 1994.42; avg error: 1.09; stdev error: 1.20
Table:  [660, 2127, 19, 26, 59, 23, 31, 25, 1730, 2619, 24, 5182, 21, 34, 2718, 23, 307, 229, 4784, 4276, 22, 171, 208, 388, 1000, 450, 173, 23, 3568, 99, 12671, 30, 25, 19, 16, 25, 33, 196, 87, 204, 9038, 2672, 3365, 271, 28, 894, 16, 63, 23, 87, 27, 398, 32, 4221, 434, 1053, 1076, 22, 2461, 222, 1080, 643, 25, 19, 21, 28, 202, 3026, 29, 31, 30, 99, 19, 23, 164, 4587, 24, 248, 18, 170, 32, 513, 34, 32, 26, 29, 4441, 5306, 2588, 1540, 1234, 372, 22, 1818, 907, 27, 353, 30, 4497, 22]
NormalizedJeroslawDPLL


100%|██████████| 100/100 [05:17<00:00,  2.83s/it]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 797.96; stdev step: 845.78; avg error: 2.99; stdev error: 1.21
Table:  [909, 682, 159, 403, 31, 398, 55, 80, 945, 3216, 127, 778, 1494, 1822, 1757, 48, 991, 229, 1520, 223, 372, 209, 505, 388, 148, 192, 139, 69, 109, 425, 4808, 93, 677, 1395, 1390, 605, 908, 3225, 468, 1227, 2139, 3135, 1885, 161, 681, 894, 19, 522, 1007, 1274, 21, 1655, 423, 2054, 164, 2449, 1311, 392, 987, 713, 64, 1243, 531, 222, 15, 397, 354, 1490, 224, 66, 2585, 84, 96, 1540, 289, 586, 466, 248, 24, 272, 109, 1048, 521, 529, 230, 1033, 1158, 588, 698, 141, 1811, 870, 507, 217, 723, 231, 835, 480, 2058, 108]


In [23]:
print_all(100, 10000, 90, light=True)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


  0%|          | 0/100 [00:00<?, ?it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 3219.85; stdev step: 4772.24; avg error: 1.60; stdev error: 1.46
Table:  [228, 30, 870, 35, 2874, 2044, 40, 69, 34, 34, 74, 11971, 18202, 156, 33, 31, 1150, 479, 8019, 66, 8182, 12393, 634, 2988, 3419, 6588, 3026, 6609, 34, 1540, 618, 6427, 1862, 36, 11318, 10242, 129, 6719, 732, 2324, 5369, 190, 1768, 28, 35, 21, 2423, 8978, 29, 28, 948, 11687, 70, 47, 904, 274, 8009, 35, 5430, 1596, 5242, 10256, 3778, 37, 18963, 40, 98, 63, 33, 28, 18521, 6918, 6808, 3301, 531, 42, 30, 3836, 28, 29, 1413, 651, 73, 41, 2441, 489, 1320, 38, 2880, 11741, 37, 33, 7867, 22321, 1698, 6486, 1462, 38, 2132, 114]
NormalizedJeroslawDPLL


100%|██████████| 100/100 [35:07<00:00, 12.62s/it]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 3388.79; stdev step: 3265.66; avg error: 3.07; stdev error: 1.57
Table:  [4636, 3256, 2119, 102, 10395, 4329, 263, 3377, 50, 251, 4941, 1741, 4528, 330, 3753, 29, 1858, 3832, 10999, 522, 3591, 1869, 29, 7651, 908, 6065, 958, 4951, 3760, 1551, 1312, 5326, 12878, 31, 8538, 2307, 129, 4107, 4832, 6749, 4757, 5746, 8753, 771, 479, 22, 2423, 10392, 117, 1971, 348, 5271, 3360, 2391, 2257, 842, 7106, 861, 5507, 619, 3027, 2498, 178, 335, 11376, 81, 1458, 1278, 1115, 723, 29, 7717, 6808, 3517, 8369, 270, 1165, 7997, 475, 4056, 5608, 971, 2987, 996, 857, 2022, 1300, 5271, 9672, 5596, 7318, 4294, 8029, 12464, 356, 5098, 291, 32, 285, 114]


In [24]:
print_all(100, 10000, 110, light=True)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


  0%|          | 0/100 [00:00<?, ?it/s]

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 20412.46; stdev step: 31339.40; avg error: 2.43; stdev error: 1.60
Table:  [38704, 51, 23735, 38, 20722, 33, 4264, 1668, 32880, 56692, 33, 32496, 35, 1436, 84083, 37, 15136, 85609, 64, 186, 96, 18064, 115595, 12862, 37, 16366, 10495, 64763, 12162, 12638, 4867, 39, 13455, 47, 627, 2936, 892, 3495, 30946, 56980, 151146, 62533, 7165, 3714, 397, 38, 742, 4010, 583, 211, 1591, 8016, 3948, 14061, 5050, 2219, 16770, 2503, 73741, 9829, 111984, 15680, 29929, 1624, 6732, 14477, 2034, 38, 15370, 56, 82586, 6893, 5109, 4146, 42, 37, 11946, 19910, 5092, 57285, 57745, 6028, 43136, 56894, 1849, 52198, 663, 34076, 4359, 135454, 10916, 730, 70605, 3824, 1296, 3657, 1934, 21303, 39, 39]
NormalizedJeroslawDPLL


100%|██████████| 100/100 [4:03:33<00:00, 86.49s/it]  

Within 1000000 steps solved 100 problems out of 100
#Sats: 100; avg step: 16790.81; stdev step: 16842.57; avg error: 3.79; stdev error: 1.37
Table:  [22619, 133, 8548, 485, 26635, 1206, 28959, 14107, 36426, 33996, 595, 31003, 35, 1436, 40184, 1429, 1959, 17620, 3625, 811, 38921, 27172, 24443, 19466, 2082, 6728, 4038, 13308, 27309, 32476, 7164, 688, 13455, 8290, 2234, 20024, 2485, 5357, 40291, 28876, 33755, 10525, 41213, 13042, 44864, 5193, 2209, 4010, 36491, 3171, 16743, 8016, 281, 59842, 107, 39, 16770, 50429, 41809, 25144, 49282, 15680, 33367, 5177, 22843, 4733, 6965, 25226, 23639, 4039, 37717, 5138, 3848, 7728, 30553, 259, 7011, 36677, 35489, 8134, 15559, 25402, 15360, 38318, 5914, 809, 1361, 8508, 15728, 97700, 552, 730, 1047, 4569, 28675, 14383, 527, 16364, 5084, 8715]


In [25]:
print_all(100, 10000, 130, light=True)

  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas
FastHumbleDPLL


 14%|█▍        | 14/100 [3:56:03<26:16:47, 1100.09s/it]

KeyboardInterrupt: 